# Non-diagonal supercell maker - python interface

First we need to compile the f90 code by Lloyd-Williams and Monserrat found here in the supplementary material: https://journals.aps.org/prb/abstract/10.1103/PhysRevB.92.184301

In [1]:
import os
topdir = os.getcwd()
print(topdir)
os.chdir('gen_sc')
!gfortran generate_supercells.f90 -o generate_supercells

/home/u2082051/phd_stuff/ndsc_tut


### NDSC interface

In [2]:
from ase.build.supercells import make_supercell
import ase.io
import subprocess
import numpy as np
import spglib

#make class for non diagonal
class NonDiagonalPhonon:
    #initialise some basic info: atoms object, a mesh grid (grid of wavevectors), path to the f90 program
    def __init__(self,a,mesh,generate_supercells):
        self.set_unit_cell(a)
        self.set_generate_supercells(generate_supercells)
        self.set_mesh(mesh)     
    def set_unit_cell(self,a):
        self.atoms = a
        self.unit_cell = (a.get_cell(),a.get_scaled_positions(),a.get_atomic_numbers())
    #find the IBZ     
    def set_mesh(self,mesh):
        self.mesh = mesh
        mapping, self.grid_full = spglib.get_ir_reciprocal_mesh(self.mesh, self.unit_cell, is_shift=[0, 0, 0])
        self.grid_full = self.grid_full / self.mesh
        mapping_unique = np.unique(mapping).tolist()
        self.grid_unique = self.grid_full[mapping_unique]    
        self.mapping = np.array([mapping_unique.index(i) for i in mapping])
        self.weights = np.array([np.count_nonzero(m==mapping) for m in mapping_unique])/len(mapping)
    
    def set_generate_supercells(self,location):
        if os.path.exists(location):
            self.generate_supercells = location
        else:
            raise FileNotFoundError("Could not find generate_supercells program at {:s}".format(location))
            
    def set_nondiagonal_supercells(self):
        with open("ibz.dat","w") as f:
            for g in self.grid_unique:
                f.write(("{:20.15f}"*3+"\n").format(*g))
        with open("grid.dat","w") as f:
            f.write(("  {:d}"*3).format(*self.mesh))
        with open("prim.dat","w") as f:
            for l in self.atoms.get_cell():
                f.write(("{:20.15f}"*3+"\n").format(*l))
        ret = subprocess.run([self.generate_supercells])#, capture_output=True)

        kpoint_to_supercell = np.loadtxt("kpoint_to_supercell.dat")
        
        self.supercells = []
        for file_index in np.sort(np.unique(kpoint_to_supercell[:,3])):
            self.supercells.append(
            np.loadtxt("supercell.{:d}.dat".format(int(file_index)))
            )
    def get_nondiagonal_supercells(self):
        supercell_atoms = []
        for s in self.supercells:
            s_copy = s * int(np.sign(np.linalg.det(s)))
            supercell_atoms.append(make_supercell(a,s_copy))
        return supercell_atoms


### Build an Atoms object or read in a file at this stage:

In [ ]:
a=ase.build.bulk('Al')

### Initialise a NonDiagonalPhonon Object: here we define the a user specified mesh density

In [4]:
ndp = NonDiagonalPhonon(a,
                        mesh=[4,4,4],
                        generate_supercells=topdir+"/gen_sc/generate_supercells")
ndp.set_nondiagonal_supercells()

### Get all the non-diagonal supercells for this mesh and store in a list

In [ ]:
all_sc = ndp.get_nondiagonal_supercells()

### Visualise the Non-diagonal supercells

In [10]:
from ase.visualize import view
view(all_sc, viewer='ase')